<b>Notebooks</b>
* `Analysis.ipynb` Main notebook for our data analysis.
* `Analysis.ipynb` Main notebook for our data analysis (median centered ones).
* `Population_Size_Estimates.ipynb` We infer populaiton size histories to fit $\beta$-SFS.
* `Highlighted_Genes.ipynb` The top 10 most significant segments tables.
* `Segmented_genome_scans.ipynb` Used to draw genomescan p-values for each super-population.


<b>Folders</b>
* `gene_scores` Estimates for each gene
* `genome_scan_plots` Plots in the paper
* `lags` Estimated Autocorrelation of statistics
* `merged` Whole genome scan results for populations
* `plots` plots of our results
* `raw` Output of our estimators

<b>Python scripts</b>
* `browser.py` Browser of our results. It can be used to access our detailed results.
* `calc_lag.py` Calculates the autocovariance of each stat
* `etas.json` Population size estimates for each population. See `Population_Size_Estimates.ipynb`.
* `gene_results.txt` Gene annotations
* `medianstats.csv` Medians for windowed stats
* `meanstats.csv` Means for windowed stats
* `popSFS.npy` Genome-wide SFS for each population
* `pvalscan.py` Calculates the p values for scan statistics
* `segmenter.py` Change Point Detection method
* `tools1kg.py` Some useful user functions to analyze 1000 genomes data

In [1]:
import pandas as pd
from browser import Pops, SPops

import numpy as np
from tools1kg import path_1000G
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
from scipy.ndimage import gaussian_filter1d
plt.style.use('ggplot')

In [2]:
Pops = {i:dict(zip(['pop', 'description'], Pops[i].split('--'))) for i in range(26)}

In [3]:
import sys
sys.path.append('/home/enes/pyslurm/') # https://github.com/enesdilber/pyslurm/
sys.path.append('/home/enes/bim/') # https://github.com/enesdilber/bim/
from pyslurm import Slurm
from utils import intersect_with_weights

In [4]:
Train = False

In [5]:
BIM = '/home/enes/bim/BIM.py'
locvars = ['Chromosome', 'start', 'end']
#sys.path.append(bim_path)

In [6]:
slurm = Slurm()
srun = slurm.batch('module load python3.8-anaconda/2020.07', 
                   '#mem-per-cpu=6000',
                   '#job-name="1kg"',
                   '#time=0-10:00:00',
                   '#cpus-per-task=1')

## Get statistics
Below calculates the several genome scan statistic including btree and bsfs. You should give the `Settings` file. It's a json file and will be red as a python dictionary. Its md5 hash will make sure the results are reproducible.

In [8]:
SFS = np.load("popSFS.npy", allow_pickle=True).item()
N = {i: np.array(SFS[i]).shape[1]+1 for i in range(26)}

In [29]:
if Train:
    gsettings = ['--eta=etas.json', '--r2t=0.001', '--r2s=0.001', '--tsz=32', '--ssz=16', '--log_pdf=logfr']
    jobs = {}
    for chrno in range(1, 23):
        for popid in range(26):     
            
            Ni = str(N[popid])
            popid = str(popid)
            chrno = str(chrno)
            job = ' '.join(['python', BIM, path_1000G(chrno), Ni, '--pop='+popid, '--out=raw/'+popid+'_'+chrno+'.csv']+gsettings)
            jobs[chrno, popid] = srun.run(job)

In [47]:
# for i in range(1,23):
#     for j in range(26):
#         if slurm.my_job_stats(jobs[str(i), str(j)])['State'] != 'COMPLETED':
#             print(i, j)

## Merge dataframes
Above code will create 26x22 csv files for each population chromosome combination, I will merge them for each population and calculate Neutrality tests.

In [8]:
z = 0

for pop in trange(26):
    z += pd.read_csv('merged/'+str(pop)+'.csv').shape[0]

In [9]:
z

8914860

In [9]:
if Train:
    def read_and_load(pop, chrno):
        df2 = pd.read_csv('raw/'+str(pop)+'_'+str(chrno)+'.csv', comment = '#')
        df2.insert(0, "Chromosome", chrno)
        return df2
    for pop in trange(26):
        pd.concat([read_and_load(pop, chrno) for chrno in range(1, 23)]).\
        to_csv('merged/'+str(pop)+'.csv', index = False)

## Change point detection
I will calculate the change points for each statitic and for each population. I will assume a fixed number of windows for each statistic and chromosome which is $n_{cp}=$ `chromosome length/1e7`. This number will give >300 regions along the chromosome, which is roughly the number of beneficial mutations along the genome.

In [82]:
if Train:
    avglen = 1e7
    jobs = {}
    for pop in range(26):
        for chrno in range(1, 23):
            job = ' '.join(['python segmenter.py', str(pop), str(chrno),str(avglen), 'raw/'])
            jobs[pop, chrno] = srun.run(job)

After the jobs are done, I merge them

In [10]:
if Train:
    
    for pop in trange(26):
        dfs = []
        for chrno in range(1, 23):
            path = 'raw/s'+str(pop)+'_'+str(chrno)+'.csv'
            dfs.append(pd.read_csv(path))

        pd.concat(dfs).to_csv('merged/s'+str(pop)+'.csv', index = False)

## Calculate Lag for each population
Because of LD all statistics have autocorrelation. In order to calculate avaraged statistic for each region we need the variance of the avaraged statistic. To calculate this variance, we need to take account the autocovariance matrix. Here we calculate it for all statistics and for all populations

In [57]:
if Train:

    stats = ['SS', 'TajD', 'Colless', 'bsfs', 'btree',
             'FulD', 'ZngE', 'FerL', 'FayH']

    jobs = []
    for stat in stats:
        job = 'python calc_lag.py ' + stat + ' 1000 merged'
        jobs.append(srun.run(job))

## Empirical p-values

In [12]:
if Train:
    
    stats = ['SS', 'TajD', 'Colless', 'bsfs', 'btree',
             'FulD', 'ZngE', 'FerL', 'FayH']

    ann = pd.read_csv('gene_result.txt', sep="\t")
    ann = ann.rename(columns = {'start_position_on_the_genomic_accession': 'start',
                                'end_position_on_the_genomic_accession': 'end'})[['chromosome', 'start', 'end', 'Symbol']]
    ann = ann[~((ann['chromosome'] == 'X') | ((ann['chromosome'] == 'Y')))]

    ann['chromosome'] = ann['chromosome'].astype('int')
    ann = ann.sort_values(['chromosome', 'start', 'end']).reset_index(drop = True)

    for pop in trange(26):
        pop = str(pop)

        df = pd.read_csv('merged/s'+pop+'.csv')
        df['start'] = df['Chromosome']+(df['start']/1e9)
        df['end'] = df['Chromosome']+(df['end']/1e9)

        bs = df['start'].to_numpy()
        be = df['end'].to_numpy()

        cps = (ann['chromosome']+(ann['start']/1e9)).to_numpy()
        cpe = (ann['chromosome']+(ann['end']/1e9)).to_numpy()

        dx = pd.DataFrame(intersect_with_weights(bs, be, cps, cpe, df[stats].to_numpy()))
        for i, stat in enumerate(stats):
            dx['ep_'+stat] = [x[i] for x in dx['val']]
            dx['ep_'+stat] = dx['ep_'+stat].rank(pct=True)

        dx.insert(0,"Chromosome", dx['start'].astype('int'))
        dx['start'] = np.round(1e9*(dx['start']-dx['Chromosome'])).astype('int')
        dx['end'] = np.round(1e9*(dx['end']-dx['Chromosome'])).astype('int')

        df = ann.rename(columns={'chromosome':'Chromosome'})
        df = df.merge(dx, on = locvars)[locvars+['Symbol']+['ep_'+stat for stat in stats]]
        df.to_csv('gene_scores/pop/'+pop+'.csv', index = False)

## Stats by the Region

In [59]:
genes = [{'Gene':'EDAR', 'pop':0},
         {'Gene':'CD5', 'pop':0},
         {'Gene':'LCT', 'pop':5},
         {'Gene':'SLC45A2', 'pop':5},
         {'Gene':'HERC2', 'pop':5},
         {'Gene':'SLC24A5', 'pop':5},
         {'Gene':'CD36', 'pop':10},
         {'Gene':'APOL1', 'pop':10}]

In [60]:
stats = ['btree', 'bsfs', 'TajD', 'Colless']
signif = 0.05
pop = ''
res = []
for gene in genes:
    
    pop = gene['pop']
    cur = gene.copy()
    cur['pop'] = Pops[pop]['pop']
    
    df = pd.read_csv('gene_scores/pop/'+str(gene['pop'])+'.csv')    
    df = df[df['Symbol'] == gene['Gene']]
    df = df[locvars+['ep_'+stat for stat in stats]]
    
    
    cur['Chromosome'] =  df['Chromosome'].iloc[0]
    cur['start'] = df['start'].iloc[0]
    cur['end'] = df['end'].iloc[-1]
    
    for stat in stats:
        if stat[:7] == 'Colless':
            cur[stat] = (df['ep_'+stat]>(1-signif)).mean()        
        else:
            cur[stat] = (df['ep_'+stat]<signif).mean()        
    
    res.append(cur)

res = pd.DataFrame(res)
res.sort_values(['pop', 'Chromosome', 'start'])

,Gene,pop,Chromosome,start,end,btree,bsfs,TajD,Colless
2,LCT,CEU,2,135787850,135837195,1.0,0.000000,0.000000,1.00
3,SLC45A2,CEU,5,33944623,33984693,0.8,0.600000,0.400000,0.60
4,HERC2,CEU,15,28111040,28322179,0.0,0.000000,0.000000,0.05
5,SLC24A5,CEU,15,48120990,48142672,0.0,0.000000,0.000000,0.00
0,EDAR,CHB,2,108894471,108989256,1.0,0.428571,0.428571,1.00
1,CD5,CHB,11,61093963,61127852,0.0,1.000000,1.000000,0.00
6,CD36,YRI,7,80602207,80679277,0.0,0.000000,0.052632,0.00
7,APOL1,YRI,22,36253071,36267531,0.0,0.000000,0.000000,0.00


### Replication of 1000 genomes selection paper

In [79]:
st2 = [{'Gene':'EDAR', 'Chromosome':2, 'start':109_510_927, 'end':109_605_828, 'pop':0},
       {'Gene':'CD5', 'Chromosome':11,'start':60_869_867, 'end':60_896_324, 'pop':0},
       {'Gene':'LCT', 'Chromosome':2,'start':136_545_410, 'end':136_594_750, 'pop':5},
       {'Gene':'SLC45A2', 'Chromosome':5,'start':33_944_721, 'end':33_984_835, 'pop':5},
       {'Gene':'HERC2', 'Chromosome':15,'start':28_356_186, 'end':28_567_298, 'pop':5},
       {'Gene':'SLC24A5', 'Chromosome':15,'start':48_413_169, 'end':48_434_869, 'pop':5},
       {'Gene':'CD36', 'Chromosome':7,'start':79_998_891, 'end':80_308_593, 'pop':10},
       {'Gene':'APOL1', 'Chromosome':22,'start':36_649_117, 'end':36_663_576, 'pop':10}]

In [80]:
stats = ['btree', 'bsfs', 'TajD', 'Colless']
signif = 0.05
pop = ''
res = []
for st in st2:
    
    if st['pop'] != pop:
        df = pd.read_csv('merged/s'+str(st['pop'])+'.csv').fillna(0)
        df = df[locvars+stats]
        for stat in stats:
            if  stat[:7] == 'Colless':
                df[stat] = -df[stat]
            df['ep_'+stat] = df[stat].rank(pct=True)<signif        

    pop = st['pop']
    chrno = st['Chromosome']
    rs, re = st['start'], st['end']
    
    nrs, nre = np.round(rs, -4), np.round(re, -4)
    re = np.where(nre>re, nre, nre+1e4).astype('int')
    rs = np.where(nrs<rs, nrs, nrs-1e4).astype('int')
    
    dx = df[df['Chromosome'] == chrno].copy().reset_index(drop = True).copy()
    dx = dx[(dx['start']>=rs) & (dx['end']<=re)]
    if not dx.shape[0]:
        dx = df[df['Chromosome'] == chrno].copy().reset_index(drop = True).copy()
        dx = dx[(dx['start']<rs) & (dx['end']>re)]
        
    
    for stat in stats:
        st['pop'] = Pops[int(pop)]['pop']
        st[stat] = dx['ep_'+stat].mean()
    res.append(st)
res = pd.DataFrame(res)
res.sort_values(['pop', 'Chromosome', 'start'])

,Gene,Chromosome,start,end,pop,btree,bsfs,TajD,Colless
2,LCT,2,136545410,136594750,CEU,1.000,0.000000,0.000000,1.000000
3,SLC45A2,5,33944721,33984835,CEU,1.000,1.000000,0.666667,1.000000
4,HERC2,15,28356186,28567298,CEU,1.000,0.882353,1.000000,1.000000
5,SLC24A5,15,48413169,48434869,CEU,1.000,1.000000,1.000000,1.000000
0,EDAR,2,109510927,109605828,CHB,0.600,0.800000,1.000000,0.400000
1,CD5,11,60869867,60896324,CHB,1.000,0.000000,1.000000,1.000000
6,CD36,7,79998891,80308593,YRI,0.125,0.053571,0.035714,0.142857
7,APOL1,22,36649117,36663576,YRI,1.000,1.000000,1.000000,1.000000


## Genome-scan p-values
Now I plot the genome-scans of all statistics for each population and save the results along with the genes

In [11]:
if Train:

    stats = ['SS', 'TajD', 'Colless', 'bsfs', 'btree',
             'FulD', 'ZngE', 'FerL', 'FayH']

    jobs = []
    for stat in stats:
        for pop in range(26):
            job = 'python pvalscan.py ' +str(pop)+' '+ stat 
            jobs.append(srun.run(job))